### Imports

In [1]:
import os
import json
from envyaml import EnvYAML
import pandas as pd

### Accessing datasets

In [2]:
def read_csv(path):
    return pd.read_csv(path)

def to_csv(df, path):
    df.to_csv(path, index=False)

E4 dataset directories

In [3]:
VAR_ENV = EnvYAML("../../env.yaml")
path_list = [os.path.join("../..", VAR_ENV["dataset.path"], VAR_ENV["dataset.version.dev"], f"P{i:02d}", "E4") for i in range(1, VAR_ENV["dataset.participants"] + 1)]

#### Dataframes

The following dataframe lists include dataframes for seperate sensors as follows.
 - `acc_df_list`: Accelerometer
 - `bvp_df_list`: Photoplethysmograph
 - `eda_df_list`: Electrodermal Activity Sensor
 - `hr_df_list`: Heart Rate                      
 - `ibi_df_list`: Inter-Beat Interval
 - `temp_df_list`: Temperature      

In [4]:
with open("../../config/e4.json" , "r") as f:
    config = json.loads(f.read())

# dataframes
for key in config["sensors"].keys():
    file_name = f"{key.upper()}.csv"
    for path in path_list:
        # dataset manipulation
        df_temp = pd.read_csv(os.path.join(path, file_name))
        initial_time = float(df_temp.columns[0])
        frequency = float(df_temp.iloc[0, 0])
        df_temp = df_temp.drop([0])
        df_temp.columns = [f"e4_{col}" for col in config["sensors"][key]["columns"]]

        # adding timestamps
        timestamps = [initial_time+(i/frequency) for i in range(len(df_temp))]
        df_temp.insert(0, "timestamp", timestamps, True) 

        # saving
        to_csv(df_temp, os.path.join(path, f"modified_{file_name}"))

Inter-Beat Interval

In [5]:
for path in path_list:
    # dataset manipulation
    df_temp = pd.read_csv(os.path.join(path, "IBI.csv"))
    initial_time = float(df_temp.columns[0])
    df = pd.DataFrame(data=[[initial_time+float(df_temp.iloc[i, 0]), 1] for i in range(len(df_temp))], columns=["timestamp", "e4_hb"]) # timestamp, heart beat
    
    # saving
    to_csv(df, os.path.join(path, "modified_IBI.csv"))